<a href="https://colab.research.google.com/github/marco-siino/PAN-CRYPTO-2023/blob/main/subtask1/training/PAN_CRYPTO_TRAINING_ELECTRA_AUG_IT_SUBTASK_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import the needed modules

In [1]:
!pip install simpletransformers
!pip install tensorflow-addons
!pip install -U deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 2

In [2]:
import numpy as np
import tensorflow_addons as tfa
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import KFold, StratifiedKFold
from google.colab import files
from deep_translator import GoogleTranslator
from urllib import request

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [6]:
# Import class Vectorizer
module_url = f"https://raw.githubusercontent.com/marco-siino/PAN-CRYPTO-2023/main/subtask1/training/crypto_vectorzer.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))
from crypto_vectorzer import Vectorizer

# Import class Dataset
module_url = f"https://raw.githubusercontent.com/marco-siino/PAN-CRYPTO-2023/main/subtask1/training/crypto_dataset.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))
from crypto_dataset import Dataset

#Import class Simulator
module_url = f"https://raw.githubusercontent.com/marco-siino/PAN-CRYPTO-2023/main/subtask1/training/simulator_kfold.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))
from simulator_kfold import Simulator

Fetching https://raw.githubusercontent.com/marco-siino/PAN-CRYPTO-2023/main/subtask1/training/crypto_vectorzer.py
Fetching https://raw.githubusercontent.com/marco-siino/PAN-CRYPTO-2023/main/subtask1/training/crypto_dataset.py
Fetching https://raw.githubusercontent.com/marco-siino/PAN-CRYPTO-2023/main/subtask1/training/simulator_kfold.py


# Load the dataset 

In [7]:
url = 'https://github.com/marco-siino/PAN-CRYPTO-2023/raw/main/subtask1/training/pan23-profiling-cryptocurrency-influencers-augmented-it.zip'
ds = Dataset(url, n_subtask = 1, language='it')
ds.build_ds(1)
xtr, xte = ds.get_train_test_df()

# CHECK SAMPLES
N = 2
for i in ds.test_set.take(N):
  print(i)

224710/224710 [==============================] - 0s 0us/step
id_label_dict {'0037a672f0ed64b3231bac64853a278d': 'nano', '03eaa72711143b521c073d9ac5745923': 'nano', '0409fe210a0edfe258d21e3404e1ce05': 'micro', '05ca545f2f700d0d5c916657251d010b': 'mega', '062492818c984febba843b650a4a602e': 'nano', '0d3700fa5c7c3fce6fd1e1ffd5282f50': 'macro', '0e1d2c43b93e8e80dc8eb6b29d48b2c1': 'no influencer', '0ed9637249db91cb2c256ec156ce1977': 'no influencer', '0f0942696ae8bcadf0db494cce7333e0': 'nano', '11727505346c5f966891f74094042073': 'nano', '1316cba6e1442a126102d5e2ab0813ec': 'macro', '13e873428de3ccfd3873b3d7258ad411': 'macro', '179aa6d5d94999ae0bf77d13bcc5fa31': 'mega', '1bc1201be24ca9d0c0afd94ca0116609': 'nano', '1bc3271b139a2c06af59e579890796f7': 'nano', '1cbf16d93cc6a672eaed9c16ac07f114': 'mega', '1fe087134fe681f8509f8d9ced432621': 'macro', '21658f2f073f0e041b79bf0f7632a0eb': 'mega', '238a39478f2f7b782dfdbeee63df503f': 'no influencer', '28ab93d1cfa1452bee4da4a93746b956': 'micro', '2a45947271

# Run Electra

In [8]:
simulator = Simulator("electra", num_fold =5, nr_epochs=30, ds=ds, vectorize_layer=['NO'], num_labels=5)
simulator.run()

Transformer Setup completed

Setup for Roberta completed.


Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.d


EPOCH NUMBER:  0

NOW TRAIN THE MODEL.


  0%|          | 0/160 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...


NOW EVALUATE THE TEST DF.


  0%|          | 0/160 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/160 [00:00<?, ?it/s]

{'mcc': 0.07285503157865601, 'acc': 0.09058201058201058, 'eval_loss': 1.605206298828125}
Macro F1 on test set is: 0.09058201058201058 


Accuracy Over epochs [0.09058201058201058]


 Over all runs maximum accuracies are: [0.09058201058201058]
RoBERTa MAX Accuracy Score on Test set ->  0.09058201058201058
RoBERTa median Accuracy Score on Test set ->  0.09058201058201058


/content/simulator_kfold.py:140: RuntimeWarning: invalid value encountered in long_scalars
  precision = TP/(TP+FP)


In [9]:
# Print a sample from the dataset.
for i in ds.test_set.take(19):
  continue

print(i)
print(i[0][0])

(<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'RT @HiNFTGiveaway: You\xe2\x80\x99ll receive 86,000,000\n\n86,000,000 in $Shib  to the first 3000 people who follows and retweets\n\n Drop your $ETH walle\xe2\x80\xa6RT @HiNFTGiveaway: You will receive 86,000,000\n\n$86,000,000 in $Shib to the first 3,000 people who follow and retweet\n\n Release your $ETH wallet... RT @TotemFi: \xf0\x9f\x92\xa0 Announcing the $SOL Wolf Pool \xf0\x9f\x92\xa0\n\n\xf0\x9f\x92\xab To celebrate our new asset pool, we are giving away:\n\n\xf0\x9f\x9a\x80 3 $SOL \xf0\x9f\x9a\x80\n\xf0\x9f\x9a\x80 $1,000 $TOTM \xf0\x9f\x9a\x80\n\nRULES:\xe2\x80\xa6RT @TotemFi: \xf0\x9f\x92\xa0 Announcement of the $SOL Wolf Pool \xf0\x9f\x92\xa0\n\n\xf0\x9f\x92\xab To celebrate our new resource pool, we\'re giving away:\n\n\xf0\x9f\x9a\x80 $3 SOL \xf0\x9f\x9a\x80\n\xf0\x9f\x9a\x80 $1,000 TOTM \xf0\x9f\x9a\x80\n\nRULES:\xe2\x80\xa6 RT @TheNFTlucky: $500 in $ETH  (5 WINNERS)\xf0\x9f\x8e\x81\xf0\x9f\x8f\x86\n\nRT + Follow me 

In [10]:
# CORRECT MAPPING LABEL-PREDICTIONS ARRAY

# ["macro" "mega" "micro" "nano" "no influencer"]

#Original order provided in random-baseline
#LABELS=['nano', 'micro', 'macro', 'mega', 'no influencer']

LABELS=['macro', 'mega', 'micro', 'nano', 'no influencer']

In [11]:
# Load the trained model.
model = ClassificationModel("electra", "outputs")

In [12]:
print(i[0].numpy()[0])
print(model.predict([str(i[0].numpy()[0])]))

new_str='The first #Crypto Project with #MoveToEarn Auto-staking &amp;amp;amp; Auto-compounding with $GMT &amp;amp;amp; $BNB Dividends Twitter- RunEarn\u2026'

print(model.predict([new_str]))
prediction = model.predict([new_str])[1]
print(prediction)
print(LABELS[np.array(prediction).argmax()])

b'RT @HiNFTGiveaway: You\xe2\x80\x99ll receive 86,000,000\n\n86,000,000 in $Shib  to the first 3000 people who follows and retweets\n\n Drop your $ETH walle\xe2\x80\xa6RT @HiNFTGiveaway: You will receive 86,000,000\n\n$86,000,000 in $Shib to the first 3,000 people who follow and retweet\n\n Release your $ETH wallet... RT @TotemFi: \xf0\x9f\x92\xa0 Announcing the $SOL Wolf Pool \xf0\x9f\x92\xa0\n\n\xf0\x9f\x92\xab To celebrate our new asset pool, we are giving away:\n\n\xf0\x9f\x9a\x80 3 $SOL \xf0\x9f\x9a\x80\n\xf0\x9f\x9a\x80 $1,000 $TOTM \xf0\x9f\x9a\x80\n\nRULES:\xe2\x80\xa6RT @TotemFi: \xf0\x9f\x92\xa0 Announcement of the $SOL Wolf Pool \xf0\x9f\x92\xa0\n\n\xf0\x9f\x92\xab To celebrate our new resource pool, we\'re giving away:\n\n\xf0\x9f\x9a\x80 $3 SOL \xf0\x9f\x9a\x80\n\xf0\x9f\x9a\x80 $1,000 TOTM \xf0\x9f\x9a\x80\n\nRULES:\xe2\x80\xa6 RT @TheNFTlucky: $500 in $ETH  (5 WINNERS)\xf0\x9f\x8e\x81\xf0\x9f\x8f\x86\n\nRT + Follow me &amp; "done"\xf0\x9f\x94\x94RT @TheNFTlucky: $500 in 

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

([1], array([[-0.07196045,  0.12164307, -0.12145996, -0.26660156,  0.07208252]]))


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

([1], array([[-0.0760498 ,  0.06536865, -0.09875488, -0.21008301,  0.01657104]]))


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[[-0.0760498   0.06536865 -0.09875488 -0.21008301  0.01657104]]
mega


In [18]:
!zip -r outputs.zip outputs

  adding: outputs/ (stored 0%)
  adding: outputs/vocab.txt (deflated 53%)
  adding: outputs/pytorch_model.bin (deflated 7%)
  adding: outputs/model_args.json (deflated 62%)
  adding: outputs/training_args.bin (deflated 49%)
  adding: outputs/config.json (deflated 56%)
  adding: outputs/special_tokens_map.json (deflated 42%)
  adding: outputs/eval_results.txt (deflated 13%)
  adding: outputs/tokenizer.json (deflated 71%)
  adding: outputs/tokenizer_config.json (deflated 42%)
  adding: outputs/checkpoint-10-epoch-1/ (stored 0%)
  adding: outputs/checkpoint-10-epoch-1/optimizer.pt (deflated 27%)
  adding: outputs/checkpoint-10-epoch-1/vocab.txt (deflated 53%)
  adding: outputs/checkpoint-10-epoch-1/pytorch_model.bin (deflated 7%)
  adding: outputs/checkpoint-10-epoch-1/model_args.json (deflated 62%)
  adding: outputs/checkpoint-10-epoch-1/training_args.bin (deflated 49%)
  adding: outputs/checkpoint-10-epoch-1/scheduler.pt (deflated 50%)
  adding: outputs/checkpoint-10-epoch-1/config.json

In [20]:
files.download('outputs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>